In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy
import warnings 
warnings.filterwarnings('ignore')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,675 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelea

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
import seaborn as sns
import numpy as np
import pandas as pd
import re
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Kkma
kkma = Kkma()

In [ ]:
movie = pd.read_csv('/content/gdrive/My Drive/data/movies.csv')
movie.drop(['Unnamed: 0'], axis = 1, inplace=True)
movie.head(5)

,title,genre,vote,vote_count,director,cast,summary,imagelink
0,포드 V 페라리,액션,관람객 평점 9.31점,"1,118",제임스 맨골드,"['맷 데이먼', '크리스찬 베일']","자존심을 건 대결의 시작!1960년대, 매출 감소에 빠진 ‘포드’는 판매 활로를 찾...",https://movie-phinf.pstatic.net/20191204_91/15...
1,감쪽같은 그녀,드라마,관람객 평점 8.83점,208,허인무,"['나문희', '김수안']",“초면에 실례하겠습니다~”\n 72살 나 홀로 라이프를 즐기는 ‘말순’ 할매의 인생...,https://movie-phinf.pstatic.net/20191204_6/157...
2,백두산,드라마,관람객 평점 7.47점,"2,828",이해준,"['이병헌', '하정우', '마동석']",대한민국 관측 역사상 최대 규모의 백두산 폭발 발생.\n 갑작스러운 재난에 한반도는...,https://movie-phinf.pstatic.net/20191219_263/1...
3,시동,드라마,관람객 평점 8.68점,"1,780",최정열,"['마동석', '박정민', '정해인']",학교도 싫고 집도 싫고 공부는 더더욱 싫다며\n '엄마'(염정아)에게 1일 1강스파...,https://movie-phinf.pstatic.net/20200117_56/15...
4,블랙머니,범죄,관람객 평점 8.75점,862,정지영,"['조진웅', '이하늬']",고발은 의무! 수사는 직진!\n할말은 하고 깔 건 깐다!일명 서울지검 ‘막프로’! ...,https://movie-phinf.pstatic.net/20191113_203/1...


결측값 확인

In [ ]:
movie.isnull().sum()

title          0
genre          0
vote          77
vote_count     0
director       0
cast           0
summary        0
imagelink      0
dtype: int64

평점이 null인 행만 추출

In [ ]:
movie[movie['vote'].isnull()][:5]

,title,genre,vote,vote_count,director,cast,summary,imagelink
49,이터널스,액션,NaN,0,클로이 자오,"['안젤리나 졸리', '마동석', '리차드 매든']","잭 커비의 코믹북을 원작으로, 수백만 년 전 인류를 실험하기 위해 지구로 온 셀레스...",https://movie-phinf.pstatic.net/20190813_248/1...
63,미친사랑,드라마,NaN,0,문시현,"['공형진', '이시원', '송재림']",과거 북파 공작원으로 활동했던 남자(두식)는 고향(남한)으로도 돌아가지 못한 채 수...,https://movie-phinf.pstatic.net/20190530_108/1...
67,아이언맨,SF,NaN,0,존 파브로,"['로버트 다우니 주니어', '테렌스 하워드', '제프 브리지스']",하이테크 슈퍼 히어로의 탄생 | 이제 업그레이드는 끝났다천재적인 두뇌와 재능으로 세...,https://movie-phinf.pstatic.net/20111222_37/13...
70,헬보이 2: 골든 아미,SF,NaN,0,길예르모 델 토로,"['론 펄먼', '셀마 블레어', '더그 존스']",잘생긴 얼굴만 세상을 구하는 건 아니지!인간과 요괴 사이에서 존재하던 고대의 휴전 ...,https://movie-phinf.pstatic.net/20111222_198/1...
74,해리 포터와 비밀의 방,판타지,NaN,0,크리스 콜럼버스,"['다니엘 래드클리프', '루퍼트 그린트', '엠마 왓슨']",해리 포터에겐 이번 여름방학이 별로 즐겁질 못했다. 마법이라면 질색을 하는 페투니아...,https://movie-phinf.pstatic.net/20111222_244/1...


vote가 null인 값들을 직접 naver 영화에서 검색해보았다. 
아직까지 개봉하지 않은 영화이거나, 개봉했으나 관람객 평점이 존재하지 않는 경우이므로 추천하는데에 무의미한 데이터라 생각하고 drop 하여 삭제하는 것으로 결정하였다.

In [ ]:
movie = movie.dropna()
movie.isnull().sum()

title         0
genre         0
vote          0
vote_count    0
director      0
cast          0
summary       0
imagelink     0
dtype: int64

In [ ]:
movie.reset_index(inplace = True)
movie.drop(['index'], axis=1, inplace = True)

줄거리를 형태소 분리하여 띄워쓰기로 구분한다.

In [ ]:
include = ['NNG','NP','NNP','NNB','NN','NX','UM','NU','VV','VA','AJ','IC']

명사와 동사, 형용사만 포함될 수 있도록 하며 한 글자로 된 단어는 제외했다. 

In [ ]:
movie['summary_processed'] = ''
for i in range(len(movie)):
  pos = kkma.pos(movie['summary'][i])
  res = set()
  for j in range(len(pos)):
    if pos[j][1] in include:
      if len(pos[j][0]) > 1:
        res.add(pos[j][0])
  movie['summary_processed'][i] = ' '.join(res)

movie.head(5)

,title,genre,vote,vote_count,director,cast,summary,imagelink,summary_processed
0,포드 V 페라리,액션,관람객 평점 9.31점,"1,118",제임스 맨골드,"['맷 데이먼', '크리스찬 베일']","자존심을 건 대결의 시작!1960년대, 매출 감소에 빠진 ‘포드’는 판매 활로를 찾...",https://movie-phinf.pstatic.net/20191204_91/15...,우승자 맞추 펼치 절대적 시간 출전 입맛 불가능 자동차 레이서 이자 활로 타협 판매...
1,감쪽같은 그녀,드라마,관람객 평점 8.83점,208,허인무,"['나문희', '김수안']",“초면에 실례하겠습니다~”\n 72살 나 홀로 라이프를 즐기는 ‘말순’ 할매의 인생...,https://movie-phinf.pstatic.net/20191204_6/157...,라이프 동생 외모 서로 시간 되어가 취향 순간 갓나 성격 벌이 나타나 못하 극인 즐...
2,백두산,드라마,관람객 평점 7.47점,"2,828",이해준,"['이병헌', '하정우', '마동석']",대한민국 관측 역사상 최대 규모의 백두산 폭발 발생.\n 갑작스러운 재난에 한반도는...,https://movie-phinf.pstatic.net/20191219_263/1...,최대 접선 시간 서울 행동 자원 규모 운명 동석 작전 전역 남과 모르 펼쳐지 성공 ...
3,시동,드라마,관람객 평점 8.68점,"1,780",최정열,"['마동석', '박정민', '정해인']",학교도 싫고 집도 싫고 공부는 더더욱 싫다며\n '엄마'(염정아)에게 1일 1강스파...,https://movie-phinf.pstatic.net/20200117_56/15...,뛰어들 최대 적수 스파이크 만나 무섭 해인 동석 포스 공부 학교 택일 뛰쳐나가 거석...
4,블랙머니,범죄,관람객 평점 8.75점,862,정지영,"['조진웅', '이하늬']",고발은 의무! 수사는 직진!\n할말은 하고 깔 건 깐다!일명 서울지검 ‘막프로’! ...,https://movie-phinf.pstatic.net/20191113_203/1...,이름 해외 최대 프로 담당 서울 누명 헐값 문제 거대 내몰리 금융 마주하 근거 일명...


vote를 평점만 남긴 데이터로 전처리한다.

In [ ]:
for i in range(len(movie)):
  vote = movie['vote'][i]
  movie['vote'][i] = float(vote[6:len(vote)-1])

movie.head(5)

,title,genre,vote,vote_count,director,cast,summary,imagelink,summary_processed
0,포드 V 페라리,액션,9.31,"1,118",제임스 맨골드,"['맷 데이먼', '크리스찬 베일']","자존심을 건 대결의 시작!1960년대, 매출 감소에 빠진 ‘포드’는 판매 활로를 찾...",https://movie-phinf.pstatic.net/20191204_91/15...,우승자 맞추 펼치 절대적 시간 출전 입맛 불가능 자동차 레이서 이자 활로 타협 판매...
1,감쪽같은 그녀,드라마,8.83,208,허인무,"['나문희', '김수안']",“초면에 실례하겠습니다~”\n 72살 나 홀로 라이프를 즐기는 ‘말순’ 할매의 인생...,https://movie-phinf.pstatic.net/20191204_6/157...,라이프 동생 외모 서로 시간 되어가 취향 순간 갓나 성격 벌이 나타나 못하 극인 즐...
2,백두산,드라마,7.47,"2,828",이해준,"['이병헌', '하정우', '마동석']",대한민국 관측 역사상 최대 규모의 백두산 폭발 발생.\n 갑작스러운 재난에 한반도는...,https://movie-phinf.pstatic.net/20191219_263/1...,최대 접선 시간 서울 행동 자원 규모 운명 동석 작전 전역 남과 모르 펼쳐지 성공 ...
3,시동,드라마,8.68,"1,780",최정열,"['마동석', '박정민', '정해인']",학교도 싫고 집도 싫고 공부는 더더욱 싫다며\n '엄마'(염정아)에게 1일 1강스파...,https://movie-phinf.pstatic.net/20200117_56/15...,뛰어들 최대 적수 스파이크 만나 무섭 해인 동석 포스 공부 학교 택일 뛰쳐나가 거석...
4,블랙머니,범죄,8.75,862,정지영,"['조진웅', '이하늬']",고발은 의무! 수사는 직진!\n할말은 하고 깔 건 깐다!일명 서울지검 ‘막프로’! ...,https://movie-phinf.pstatic.net/20191113_203/1...,이름 해외 최대 프로 담당 서울 누명 헐값 문제 거대 내몰리 금융 마주하 근거 일명...


vote_count 또한 int형으로 바꾸어준다..

In [ ]:
for i in range(len(movie)):
  if type(movie['vote_count'][i]) != int and ',' in movie['vote_count'][i]:
    a = list(movie['vote_count'][i])
    a.pop(a.index(','))
    a = ''.join(a)
    movie['vote_count'][i] = int(a)
  else:
    movie['vote_count'][i] = int(movie['vote_count'][i])

가중 평점 = (v/(v+m)) R + (m/(v+m)) \ C

* v: 개별 영화에 평점을 투표한 횟수
* m: 평점을 부여하기 위한 최소 투표 횟수
* R: 개별 영화에 대한 평균 평점
* C: 전체 영화에 대한 평균 평점


v 는 'vote_count' 값이며, R은 'vote_average' 값에 해당합니다. C의 경우 전체 영화의 평균 평점이므로 movies['ratings'].mean()으로 구할 수 있습니다.
m 의 경우는 투표 횟수에 따른 가중치를 직접 조절하는 역할을 하는데, m을 높이면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 주게 됩니다.
여기서는 m을 상위 60% 해당하는 횟수를 기준으로 정하겠습니다.

In [ ]:
C = movie['vote'].mean()
m = movie['vote_count'].quantile(0.6)
print('C:',round(C,3),'m:',round(m,3))

C: 8.465 m: 312.4


In [ ]:
percentile = 0.6
C = movie['vote'].mean()
m = movie['vote_count'].quantile(0.6)

def weighted_vote_average(record):
  v = record['vote_count']
  R = record['vote']

  return ((v/(v+m)) * R) +( (m/(v+m)) * C)

movie['weighted_vote'] = movie.apply(weighted_vote_average, axis = 1)
movie.head(5)

,title,genre,vote,vote_count,director,cast,summary,imagelink,summary_processed,weighted_vote
0,포드 V 페라리,액션,9.31,1118,제임스 맨골드,"['맷 데이먼', '크리스찬 베일']","자존심을 건 대결의 시작!1960년대, 매출 감소에 빠진 ‘포드’는 판매 활로를 찾...",https://movie-phinf.pstatic.net/20191204_91/15...,우승자 맞추 펼치 절대적 시간 출전 입맛 불가능 자동차 레이서 이자 활로 타협 판매...,9.125478
1,감쪽같은 그녀,드라마,8.83,208,허인무,"['나문희', '김수안']",“초면에 실례하겠습니다~”\n 72살 나 홀로 라이프를 즐기는 ‘말순’ 할매의 인생...,https://movie-phinf.pstatic.net/20191204_6/157...,라이프 동생 외모 서로 시간 되어가 취향 순간 갓나 성격 벌이 나타나 못하 극인 즐...,8.610960
2,백두산,드라마,7.47,2828,이해준,"['이병헌', '하정우', '마동석']",대한민국 관측 역사상 최대 규모의 백두산 폭발 발생.\n 갑작스러운 재난에 한반도는...,https://movie-phinf.pstatic.net/20191219_263/1...,최대 접선 시간 서울 행동 자원 규모 운명 동석 작전 전역 남과 모르 펼쳐지 성공 ...,7.568992
3,시동,드라마,8.68,1780,최정열,"['마동석', '박정민', '정해인']",학교도 싫고 집도 싫고 공부는 더더욱 싫다며\n '엄마'(염정아)에게 1일 1강스파...,https://movie-phinf.pstatic.net/20200117_56/15...,뛰어들 최대 적수 스파이크 만나 무섭 해인 동석 포스 공부 학교 택일 뛰쳐나가 거석...,8.647918
4,블랙머니,범죄,8.75,862,정지영,"['조진웅', '이하늬']",고발은 의무! 수사는 직진!\n할말은 하고 깔 건 깐다!일명 서울지검 ‘막프로’! ...,https://movie-phinf.pstatic.net/20191113_203/1...,이름 해외 최대 프로 담당 서울 누명 헐값 문제 거대 내몰리 금융 마주하 근거 일명...,8.674220


* 가중 평점을 적용하지 않은 평점 순위 Top 10

In [ ]:
movie[['title','vote','weighted_vote','vote_count']].sort_values('vote', ascending=False)[:10]

,title,vote,weighted_vote,vote_count
101,러브 앳,10,8.479719,3
186,앨리스 죽이기,10,8.474884,2
262,체 게바라: 2부 게릴라,10,8.470017,1
224,집으로...,10,8.498758,7
273,쿵푸 보이,10,8.479719,3
194,발리: 천상의 울림,10,8.470017,1
287,너의 목소리,10,8.474884,2
260,퍼펙트 타겟,10,8.470017,1
191,물의 기억,10,8.479719,3
7,사선의 끝,10,8.474884,2


* 가중 평점을 적용한 Top 10

In [ ]:
movie[['title','vote','weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote,weighted_vote,vote_count
326,어벤져스: 엔드게임,9.5,9.490055,32196
38,보헤미안 랩소디,9.45,9.427144,13149
34,그린 북,9.55,9.407861,2072
50,알라딘,9.42,9.386280,8534
37,가버나움,9.54,9.343216,1394
31,항거:유관순 이야기,9.39,9.254694,1823
231,스파이더맨: 뉴 유니버스,9.38,9.247214,1840
315,위대한 쇼맨,9.31,9.247133,3886
283,뽀로로 극장판 보물섬 대모험,9.35,9.223276,1869
217,극한직업,9.2,9.185950,16028


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df = 0, ngram_range=(1,2))
summ_mat = count_vect.fit_transform(movie['summary_processed'])
print(summ_mat.shape)

(334, 14748)


In [ ]:
summ_sim = cosine_similarity(summ_mat, summ_mat)
summ_sim_sorted_ind = summ_sim.argsort()[:,::-1] #유사도가 높은 순으로 정리된 summ_sim 객체의 비교 행 위치 값을 얻을 수 있습니다.

추출해냈던 `summary_processed`에 대해 코사인 유사도를 구하여 줄거리 키워드가 유사한 영화 상위 10개를 추천해보자

In [ ]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 df에서 'title' 칼럼이 입력된 title_name 값인 데이터프레임 추출
    title_movie = df[df['title'] == title_name]

    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1) #1차원 array로 변경

    similar_indexes = similar_indexes[similar_indexes != title_index]

    return df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n]

In [ ]:
movie[movie['title'] == '어벤져스: 엔드게임'][['title','vote','weighted_vote','summary_processed']]

,title,vote,weighted_vote,summary_processed
326,어벤져스: 엔드게임,9.5,9.490055,희망 위대 지구 떠나 이후 전쟁 살아남 마지막 최후 운명 바꾸 절반 위하 펼쳐지


In [ ]:
similar_movies = find_sim_movie(movie, summ_sim_sorted_ind, '어벤져스: 엔드게임', 10)
similar_movies[['title','vote','weighted_vote','summary_processed']]

,title,vote,weighted_vote,summary_processed
59,토이 스토리 4,9.23,9.151045,여행 새롭 사라지 친구 위험천만 장난감 떠나 버즈 만나 우리 거부 끝나 운명 나서 ...
297,언더독,9.47,9.006574,적응 라이프 찰나 무리 떠나 만나 댕댕이 운명 고참 자유 꿈꾸 구하 뭉치 위하 역전...
293,드래곤볼 슈퍼: 브로리,9.47,8.847628,전투 무기력 운명 뛰어넘 시도 나타나 처절 대회 엄청나 돌아오 되찾 오공 상상 최후...
240,다크 나이트,9.65,8.617197,반장 펼치 고담 운명 작전 조직 내던지 악당 준비 제거 어리 조커 지키 최후 매이 ...
158,존 윅 3: 파라벨룸,8.64,8.604717,모든 국제 파문 총구 출구 킬러 베리 준비 보호 내려지 찾아가 전설 위하 시시각각 ...
254,아일라,9.13,8.586829,이름 칠흑 아이 서로 명령 이야기 포화 운명 발견 감동 실화 한국 부모 존재 감행 ...
327,터미네이터 2:오리지널,9.51,8.581085,인간 코너 없애 미래 운명 과거 어리 스카이 지키 로봇 위하 시작하 가쁘 사령관 금...
328,터미네이터 2,9.51,8.581085,인간 코너 없애 미래 운명 과거 어리 스카이 지키 로봇 위하 시작하 가쁘 사령관 금...
278,"안녕, 티라노: 영원히, 함께",8.6,8.511366,펼쳐지 이제 사랑 천국 니야 약속 희망 아프 떠나 관하 이야기 비밀 우정 간직 가슴 혼자
261,체 게바라: 1부 아르헨티나,9.33,8.473346,펼치 문인 점령 민중 작전 고통 최후 감행 게릴라 군부 위하 카스트 독재 마지막 아...


In [ ]:
movie[movie['title'] == '보헤미안 랩소디'][['title','vote','weighted_vote','summary_processed']]

,title,vote,weighted_vote,summary_processed
38,보헤미안 랩소디,9.45,9.427144,이름 시간 몰르 월드 일하 동안 이야기 퍼포먼스 우리 수하물 노동자 스타 독창적 외...


In [ ]:
similar_movies = find_sim_movie(movie, summ_sim_sorted_ind, '보헤미안 랩소디', 10)
similar_movies[['title','vote','weighted_vote','summary_processed']]

,title,vote,weighted_vote,summary_processed
231,스파이더맨: 뉴 유니버스,9.38,9.247214,평행 피터 모든 열리 결성 유니 만나 우리 스파이 마주치 가지 누아르 악당 물리 방...
182,브링 더 소울 : 더 무비,9.68,9.074111,시간 서울 소년단 이야기 둘르 뒤풀이 방탄 펼쳐지 풀어지 전세계 콘서트 끝나 열광 ...
272,너의 이름은.,9.02,9.004857,이름 흘러가 서로 시간 되어가 다가오 만나 우리 운명 혜성 메모 이상 뒤바뀌 낯설 ...
133,마이펫의 이중생활2,9,8.854875,사랑 라이프 생각 우리 영혼 집구석 시크 로이 완벽 모르 고프 자유 걱정 편하 스노...
145,범블비,8.75,8.706361,이름 인간 목격 서로 되어가 비의 이야기 자동차 발견 거대 가지 몰리 모르 광경 프...
202,위로공단,9.02,8.657658,이름 신음 담기 마니 오늘날 일하 이야기 여정 우리 오늘 독창적 과거 변모 감정 어...
67,왕이 될 아이,9.1,8.639181,엑스 미래 루이스 스튜 발견 포스 학교 레베카 나타나 마법사 절대악 꽂히 군단 바위...
269,신비아파트 극장판 하늘도깨비 대 요르문간드,8.72,8.624801,떠나 파괴 왕국 순간 거세 바닷속 발견 마사 비행기 남매 여행 겨울 하늘 깨비 합류...
254,아일라,9.13,8.586829,이름 칠흑 아이 서로 명령 이야기 포화 운명 발견 감동 실화 한국 부모 존재 감행 ...
21,로켓맨,8.8,8.585797,사랑 뒤흔들 상처 배신 순간 발매 퍼포먼스 독보적 대하 자리잡 노래 연잇 아이콘 진...


줄거리에 나타나는 키워드 유사도를 바탕으로 추천해보았을 때엔 `나름대로..` 비슷한 키워드가 포함된 영화들이 추천되는 것 같다. 물론 엄청난 관련은 없어보이지만... 

이제는 좀 더 세부적인 조건을 추가해보려고 한다.
* 장르가 같으면 가산점
* 감독이 같으면 가산점
* 출연하는 배우가 같으면 가산점

---

### 장르가 같으면 가산점
장르는 기본적으로 포함하여 줄거리 유사도와 장르의 일치여부를 판단하여 추천해보려고 한다. 

### 감독, 출연하는 배우가 같으면 가산점
이 부분은 `선택`으로 줄 생각이다. 만약 입력되지 않는다면 장르와 줄거리 유사도를 기반으로 10가지 영화를 추천하는 반면에, 감독이나 배우가 입력된다면 유사도의 우선순위를 `장르 > 감독 or 배우 > 줄거리유사도` 순으로 주고, 입력되지 않는다면 유사도 우선순위를 `장르 > 줄거리유사도` 순으로 준다.


이는 해당 영화를 좋아한다고 해서 그 영화의 `감독`을 좋아하진 않는다는 가정을 바탕으로 진행하며 `배우` 또한 마찬가지이다. 

앞서 구현했던 find_sim_movie를 장르를 포함할 수 있도록 수정해보자.
대신, 동일한 장르의 영화만 10가지 모두 추천되지 않도록 같은 장르의 영화는 50% 추천될 수 있도록 해보자.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
def get_similarity(df):
  count_vect = CountVectorizer(min_df = 0, ngram_range=(1,2))
  mat = count_vect.fit_transform(df['summary_processed'])

  sim = cosine_similarity(mat, mat)
  sim_sorted_ind = sim.argsort()[:,::-1]
  return mat, sim, sim_sorted_ind

In [ ]:
summ_mat, summ_sim, summ_sim_sorted_ind = get_similarity(movie)

1. 장르가 일치하는 것에서 줄거리 유사도를 산충해서 높은 것 5가지를 뽑는다.
2. 줄거리 유사도가 일치하는 것들 중 1과 겹치지 않는 것을 추가한다.

In [ ]:
def find_sim_movie(df, summ_sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 df에서 'title' 칼럼이 입력된 title_name 값인 데이터프레임 추출
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    #먼저, 장르가 일치하는 것들에 대한 dataframe을 구한다.
    genre = df[df['title'] == title_name]['genre']
    genre_name = genre.values[0]

    print("추천 기준 영화 정보")
    print(df[df['title'] == title_name][['title','genre','summary_processed']])

    genre_df = df[df['genre'] == genre_name]
    genre_df.reset_index(inplace = True)
    genre_df.drop(['index'], axis=1, inplace = True)
    
    g_mat, g_sim, g_sim_sorted_ind = get_similarity(genre_df)
    g_title_movie = genre_df[genre_df['title'] == title_name]
    g_title_index = g_title_movie.index.values
    g_similar_indexes = g_sim_sorted_ind[g_title_index, :(top_n)]
    g_similar_indexes = g_similar_indexes.reshape(-1)
    g_similar_indexes = g_similar_indexes[g_similar_indexes != g_title_index]
    g = genre_df.iloc[g_similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n//2]

# /    줄거리가 유사한 것들에 대해 다시 진행
    similar_indexes = summ_sorted_ind[title_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1) #1차원 array로 변경
    similar_indexes = similar_indexes[similar_indexes != title_index]
    s = df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n//2]

    # 혹시 모를 후보군
    candidate = df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n//2]

    result = pd.concat([g, s])
    result = result.sort_values('weighted_vote', ascending=False)
    result.reset_index(inplace = True)
    result.drop(['index'],axis=1,inplace=True)
    result = result.drop_duplicates(['title'], keep='first') #중복 제거

    if len(result) < 10: #열개보다 적은 경우
      result = pd.concat([result, candidate]) #후보군과 먼저 합친다.
      result = result.drop_duplicates(['title'], keep='first') #다시 중복을 제거하고
      result = result[:top_n] #열개만큼 잘라낸 뒤,
      result = result.sort_values('weighted_vote',ascending=False) #다시 별점 순으로 변경
    return result

In [ ]:
similar_movies = find_sim_movie(movie, summ_sim_sorted_ind, '어벤져스: 엔드게임', 10)
similar_movies[['title','vote','weighted_vote','genre','summary_processed']]

추천 기준 영화 정보
          title genre                             summary_processed
326  어벤져스: 엔드게임    액션  희망 위대 지구 떠나 이후 전쟁 살아남 마지막 최후 운명 바꾸 절반 위하 펼쳐지


,title,vote,weighted_vote,genre,summary_processed
0,토이 스토리 4,9.23,9.151045,애니메이션,여행 새롭 사라지 친구 위험천만 장난감 떠나 버즈 만나 우리 거부 끝나 운명 나서 ...
1,어벤져스: 인피니티 워,9.08,9.067384,액션,펼쳐지 최강 목격 조합 역대 걸리 스톤 세계 무한 운명 향하 대결 마블 클라이맥스 이루
2,드래곤볼 슈퍼: 브로리,9.47,8.847628,애니메이션,전투 무기력 운명 뛰어넘 시도 나타나 처절 대회 엄청나 돌아오 되찾 오공 상상 최후...
3,다크 나이트,9.65,8.617197,액션,반장 펼치 고담 운명 작전 조직 내던지 악당 준비 제거 어리 조커 지키 최후 매이 ...
5,존 윅 3: 파라벨룸,8.64,8.604717,액션,모든 국제 파문 총구 출구 킬러 베리 준비 보호 내려지 찾아가 전설 위하 시시각각 ...
6,아일라,9.13,8.586829,드라마,이름 칠흑 아이 서로 명령 이야기 포화 운명 발견 감동 실화 한국 부모 존재 감행 ...
7,"안녕, 티라노: 영원히, 함께",8.6,8.511366,애니메이션,펼쳐지 이제 사랑 천국 니야 약속 희망 아프 떠나 관하 이야기 비밀 우정 간직 가슴 혼자
8,캡틴 마블,8.45,8.450464,액션,살아가 희망 위협 크리 요원 사무엘 발견 감지 공군 합치 캐럴 버스 위기 불시착 빠...
9,안나,8.2,8.402304,액션,킬러 모든 코어 위협 루스 제거 하드 시작 미션 마지막 살아남 액션 모델 파리 위하 위장


In [ ]:
similar_movies = find_sim_movie(movie, summ_sim_sorted_ind, '보헤미안 랩소디', 10)
similar_movies[['title','vote','weighted_vote','genre','summary_processed']]

추천 기준 영화 정보
       title genre                                  summary_processed
38  보헤미안 랩소디   드라마  이름 시간 몰르 월드 일하 동안 이야기 퍼포먼스 우리 수하물 노동자 스타 독창적 외...


,title,vote,weighted_vote,genre,summary_processed
0,스파이더맨: 뉴 유니버스,9.38,9.247214,애니메이션,평행 피터 모든 열리 결성 유니 만나 우리 스파이 마주치 가지 누아르 악당 물리 방...
1,너의 이름은.,9.02,9.004857,애니메이션,이름 흘러가 서로 시간 되어가 다가오 만나 우리 운명 혜성 메모 이상 뒤바뀌 낯설 ...
2,생일,9.01,8.902019,드라마,아들 살아가 서로 떠나 이야기 만나 우리 생일 간직 돌아오 가족 남기 정일 특별 이...
3,위로공단,9.02,8.657658,다큐멘터리,이름 신음 담기 마니 오늘날 일하 이야기 여정 우리 오늘 독창적 과거 변모 감정 어...
4,신비아파트 극장판 하늘도깨비 대 요르문간드,8.72,8.624801,애니메이션,떠나 파괴 왕국 순간 거세 바닷속 발견 마사 비행기 남매 여행 겨울 하늘 깨비 합류...
5,아일라,9.13,8.586829,드라마,이름 칠흑 아이 서로 명령 이야기 포화 운명 발견 감동 실화 한국 부모 존재 감행 ...
6,로켓맨,8.8,8.585797,드라마,사랑 뒤흔들 상처 배신 순간 발매 퍼포먼스 독보적 대하 자리잡 노래 연잇 아이콘 진...
8,쉰들러 리스트,9.3,8.580307,드라마,적히 인건비 서서 그곳 독일 도시 이자 점령 방법 독일인 나치 용기 기회 폴란드 회...
9,미성년,8.64,8.574603,드라마,관심 어떻 동안 이야기 만나 폭풍 주리 수습 소진 학교 모르 엄청나 불러오 아빠 미...


In [ ]:
similar_movies = find_sim_movie(movie, summ_sim_sorted_ind, '알라딘', 10)
similar_movies[['title','vote','weighted_vote','genre','summary_processed']]

추천 기준 영화 정보
   title genre                                  summary_processed
50   알라딘    모험  소원 사막 아그라 생각 왕국 만나 가지 마법사 들어주 자파 좀도둑 마음 의뢰 자스민...


,title,vote,weighted_vote,genre,summary_processed
0,토이 스토리 4,9.23,9.151045,애니메이션,여행 새롭 사라지 친구 위험천만 장난감 떠나 버즈 만나 우리 거부 끝나 운명 나서 ...
1,레드슈즈,9.14,8.938324,애니메이션,이전 끊이 난쟁이 신고 희망 생각 왕국 운명 아름다움 구두 완벽 초록색 머물 사고 ...
2,겨울왕국 2,8.95,8.926177,애니메이션,두려워하 스토프 위협 떠나 왕국 만나 운명 과거 목소리 알려주 나가 진실 조언 누구...
3,왕이 될 아이,9.1,8.639181,모험,엑스 미래 루이스 스튜 발견 포스 학교 레베카 나타나 마법사 절대악 꽂히 군단 바위...
4,라이온 킹,8.64,8.622503,모험,희망 죽음 날르 떠나 왕국 어린 만나 대하 삼촌 과거 마주하 용기 의욕 야심 되찾 ...
5,시인 할매,9.53,8.558421,다큐멘터리,박사 이름 소원 모질 세월 시집살이 전부 할매 위로 만나 가족 만이 서럽 석자 배우...
6,위시업,9,8.468522,뮤지컬,일은 소원 시기 모든 음색 운동 순간 방법 발견 스타 노래 나타나 니르 즐기 들어주...
7,필그리미지,7.8,8.454642,모험,위협 한창 십자군 로마 노리 가지 벙어리 나타나 수도사 이르 지키 내고 수도원 위하...
8,신비한 동물들과 그린델왈드의 범죄,7.46,7.595966,모험,장담 운명 활약 야욕 알버스 마법사 순혈 세력 레드 합중국 모으 위하 탈출 분열 사...


### 감독명과 배우명 추가하기

* 감독만 입력으로 들어오는 경우  
`감장줄 5 감장 3 감줄 2 감 1`
* 배우만 입력으로 들어오는 경우  
`배장줄 5 배장2 배줄 2 배 1`
* 감독 & 배우가 입력으로 들어오는 경우  
`감배장줄 5 감배장 2 감배줄2 감배1`   
  * 만약, 감독&배우가 겹치는 영화가 5개 미만인 경우  
  `감장줄2, 배장줄2 > 감배장2 없으면 감장 + 배장 2> 감배줄유 2 없으면 감줄 + 배줄 2 > 감배 2`

In [ ]:
movie.head(5)

,title,genre,vote,vote_count,director,cast,summary,imagelink,summary_processed,weighted_vote
0,포드 V 페라리,액션,9.31,1118,제임스 맨골드,"['맷 데이먼', '크리스찬 베일']","자존심을 건 대결의 시작!1960년대, 매출 감소에 빠진 ‘포드’는 판매 활로를 찾...",https://movie-phinf.pstatic.net/20191204_91/15...,우승자 맞추 펼치 절대적 시간 출전 입맛 불가능 자동차 레이서 이자 활로 타협 판매...,9.125478
1,감쪽같은 그녀,드라마,8.83,208,허인무,"['나문희', '김수안']",“초면에 실례하겠습니다~”\n 72살 나 홀로 라이프를 즐기는 ‘말순’ 할매의 인생...,https://movie-phinf.pstatic.net/20191204_6/157...,라이프 동생 외모 서로 시간 되어가 취향 순간 갓나 성격 벌이 나타나 못하 극인 즐...,8.610960
2,백두산,드라마,7.47,2828,이해준,"['이병헌', '하정우', '마동석']",대한민국 관측 역사상 최대 규모의 백두산 폭발 발생.\n 갑작스러운 재난에 한반도는...,https://movie-phinf.pstatic.net/20191219_263/1...,최대 접선 시간 서울 행동 자원 규모 운명 동석 작전 전역 남과 모르 펼쳐지 성공 ...,7.568992
3,시동,드라마,8.68,1780,최정열,"['마동석', '박정민', '정해인']",학교도 싫고 집도 싫고 공부는 더더욱 싫다며\n '엄마'(염정아)에게 1일 1강스파...,https://movie-phinf.pstatic.net/20200117_56/15...,뛰어들 최대 적수 스파이크 만나 무섭 해인 동석 포스 공부 학교 택일 뛰쳐나가 거석...,8.647918
4,블랙머니,범죄,8.75,862,정지영,"['조진웅', '이하늬']",고발은 의무! 수사는 직진!\n할말은 하고 깔 건 깐다!일명 서울지검 ‘막프로’! ...,https://movie-phinf.pstatic.net/20191113_203/1...,이름 해외 최대 프로 담당 서울 누명 헐값 문제 거대 내몰리 금융 마주하 근거 일명...,8.674220


In [ ]:
# 감독, 배우가 일치하는 것들에 대한 dataframe을 뽑기
director_name = '봉준호'

# 감독이 일치하는 데이터프레임
director = movie[movie['director'] == director_name]
director.head()

,title,genre,vote,vote_count,director,cast,summary,imagelink,summary_processed,weighted_vote
16,기생충,드라마,9.07,7913,봉준호,"['송강호', '이선균', '조여정']",“폐 끼치고 싶진 않았어요”전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) ...,https://movie-phinf.pstatic.net/20190528_36/15...,장남 명문대 자리 희망 저택 걷잡 싹트 여정 기다리 이렇 강호 과외 맞이하 가족 사...,9.047027
245,살인의 추억,범죄,9.65,17,봉준호,"['송강호', '김상경']","연쇄살인 실화극 | 1986년 시골마을, 두 형사에겐 모든 것이 처음이었다. | 당...",https://movie-phinf.pstatic.net/20111222_166/1...,시체 토박이 박두 후임 수법 발견 벌이 여자 시경 실화 수사 공포 강박 서류 필두 ...,8.526270
246,마더,범죄,9.45,11,봉준호,"['김혜자', '원빈']","아들의 살인혐의, 엄마의 사투 | 아무도 믿지마 엄마가 구해줄께읍내 약재상에서 일하...",https://movie-phinf.pstatic.net/20111223_73/13...,아들 무도 굳어지 준의 일하 해지 당하 경찰 몰리 사고 태우 살인 애간장 백방 구하...,8.498619


In [ ]:
cast_name = '마동석'

indexes = []
for i in range(len(movie)):
  if cast_name in movie['cast'][i]:
    indexes.append(i)

cast = movie.iloc[indexes]
cast.head()

,title,genre,vote,vote_count,director,cast,summary,imagelink,summary_processed,weighted_vote
2,백두산,드라마,7.47,2828,이해준,"['이병헌', '하정우', '마동석']",대한민국 관측 역사상 최대 규모의 백두산 폭발 발생.\n 갑작스러운 재난에 한반도는...,https://movie-phinf.pstatic.net/20191219_263/1...,최대 접선 시간 서울 행동 자원 규모 운명 동석 작전 전역 남과 모르 펼쳐지 성공 ...,7.568992
3,시동,드라마,8.68,1780,최정열,"['마동석', '박정민', '정해인']",학교도 싫고 집도 싫고 공부는 더더욱 싫다며\n '엄마'(염정아)에게 1일 1강스파...,https://movie-phinf.pstatic.net/20200117_56/15...,뛰어들 최대 적수 스파이크 만나 무섭 해인 동석 포스 공부 학교 택일 뛰쳐나가 거석...,8.647918
303,나쁜 녀석들: 더 무비,범죄,8.16,2013,손용호,"['마동석', '김상중', '김아중']",죄수들이 탈출했다!\n 교도소 호송차량이 전복되고 최악의 범죄자들이 탈주하는 사상 ...,https://movie-phinf.pstatic.net/20190910_213/1...,반장 차량 생각 오구 죄수 사기꾼 고유성 행동 활약 과거 경찰 동석 전직 거대 조직...,8.200991
304,성난황소,범죄,8.66,2667,김민호,"['마동석', '송지효']",거칠었던 과거를 벗어나 수산시장에서 건어물 유통을 하며 건실하게 살던 동철(마동석)...,https://movie-phinf.pstatic.net/20181031_241/1...,신고 아내 성오 거칠 과거 동석 경찰 시장 건드리 오지 걸리 건어물 행방 거액 납치...,8.639566
306,악인전,범죄,8.55,2103,이원태,"['마동석', '김무열', '김성규']",우연히 연쇄살인마의 표적이 되었다 살아난 조직 보스 장동수와\n 범인잡기에 혈안이 ...,https://movie-phinf.pstatic.net/20190515_203/1...,손잡 보스 표적 살아나 사람 범인 태석 강력 연쇄 위하 살인마 미친개 동수 혈안 타...,8.539022
